In [437]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
from itertools import accumulate
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [438]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


In [439]:
#cities : list of city names
#city_indices : list of (city: index)
cities = CITIES['name'].values.tolist()
city_indices = {city: index for index, city in enumerate(cities)}
#city_indices['Bari']

In [440]:
#returns for a route the cost 
def cost(route):
    route.append(route[0])
    #assert set(route) == set(range(len(CITIES)))
    cost = 0
    for c1, c2 in zip(route, route[1:]):
        cost += DIST_MATRIX[city_indices[c1], city_indices[c2]]
        #print(city_indices[c1], c1, city_indices[c2], c2, DIST_MATRIX[city_indices[c1], city_indices[c2]])
    route.pop()
    return cost

def fitness(route):
    return -cost(route)

  

In [441]:
def greedy_route():
    current_city = random.choice(cities)
    route = [current_city]
    not_visited = set(cities) - {current_city}
    
    while not_visited:
        closest = min(not_visited, key=lambda city: DIST_MATRIX[city_indices[current_city], city_indices[city]])
        route.append(closest)
        not_visited.remove(closest)
        current_city = closest
        
    return route

def initial_population(size):
    population = [greedy_route() for i in range(size)]
    return population


In [ ]:
#returns a random population of routes, leads to a worse result
def initial_population2(size):
    population = set()
    while len(population) < size:
        route = tuple(random.sample(CITIES['name'].values.tolist(), len(CITIES)))
        population.add(route)

    # Convert each route back to a list
    return [list(route) for route in population]

In [442]:
pop = initial_population(10)
print(pop)

#for route in pop:
    #print(fitness(route))
    
print(fitness(pop[0]))

[['Arkhangelsk', 'Severodvinsk', 'Petrozavodsk', 'Saint Petersburg', 'Kolpino', 'Velikiy Novgorod', 'Pskov', 'Velikie Luki', 'Smolensk', 'Bryansk', 'Oryol', 'Kursk', 'Staryy Oskol', 'Voronezh', 'Lipetsk', 'Yelets', 'Novomoskovsk', 'Tula', 'Serpukhov', 'Obninsk', 'Kaluga', 'Podolsk', 'Odintsovo', 'Moscow', 'Mytishchi', 'Korolyov', 'Shchyolkovo', 'Balashikha', 'Zheleznodorozhnyy', 'Lyubertsy', 'Zhukovskiy', 'Elektrostal', 'Noginsk', 'Orekhovo‐Zuevo', 'Sergiyev Posad', 'Khimki', 'Zelenograd', 'Tver', 'Rybinsk', 'Yaroslavl', 'Kostroma', 'Ivanovo', 'Kovrov', 'Vladimir', 'Murom', 'Arzamas', 'Dzerzhinsk', 'Nizhny Novgorod', 'Cheboksary', 'Novocheboksarsk', 'Yoshkar‐Ola', 'Kazan', 'Ulyanovsk', 'Dimitrovgrad', 'Tolyatti', 'Novokuybyshevsk', 'Samara', 'Syzran', 'Balakovo', 'Engels', 'Saratov', 'Kamyshin', 'Volzhskiy', 'Volgograd', 'Volgodonsk', 'Shakhty', 'Novoshakhtinsk', 'Novocherkassk', 'Rostov‐na‐Donu', 'Bataysk', 'Taganrog', 'Krasnodar', 'Maykop', 'Armavir', 'Stavropol', 'Nevinnomyssk', 'Ch

In [444]:
population_size = 5
population = initial_population(10)
print(population)

# route_cost = []
# for i in range(len(population)):
#     route_cost.append(cost(population[i]))

# print(min(route_cost))


[['Tver', 'Zelenograd', 'Khimki', 'Mytishchi', 'Korolyov', 'Shchyolkovo', 'Balashikha', 'Zheleznodorozhnyy', 'Lyubertsy', 'Moscow', 'Odintsovo', 'Podolsk', 'Zhukovskiy', 'Elektrostal', 'Noginsk', 'Orekhovo‐Zuevo', 'Sergiyev Posad', 'Vladimir', 'Kovrov', 'Ivanovo', 'Kostroma', 'Yaroslavl', 'Rybinsk', 'Cherepovets', 'Vologda', 'Dzerzhinsk', 'Nizhny Novgorod', 'Arzamas', 'Murom', 'Ryazan', 'Kolomna', 'Serpukhov', 'Obninsk', 'Kaluga', 'Tula', 'Novomoskovsk', 'Yelets', 'Lipetsk', 'Voronezh', 'Staryy Oskol', 'Belgorod', 'Kursk', 'Oryol', 'Bryansk', 'Smolensk', 'Velikie Luki', 'Pskov', 'Velikiy Novgorod', 'Kolpino', 'Saint Petersburg', 'Petrozavodsk', 'Severodvinsk', 'Arkhangelsk', 'Murmansk', 'Syktyvkar', 'Kirov', 'Yoshkar‐Ola', 'Novocheboksarsk', 'Cheboksary', 'Kazan', 'Ulyanovsk', 'Dimitrovgrad', 'Tolyatti', 'Novokuybyshevsk', 'Samara', 'Syzran', 'Balakovo', 'Engels', 'Saratov', 'Kamyshin', 'Volzhskiy', 'Volgograd', 'Volgodonsk', 'Shakhty', 'Novoshakhtinsk', 'Novocherkassk', 'Rostov‐na‐Don

In [445]:
def selected_parents(population):
    
    routes_fitnesses = [(route, fitness(route)) for route in population]
    # print("\n fitnesses")
    # print(routes_fitnesses)
    #print("max fitness:", max(route_fitnesses), "- index:", route_fitnesses.index(max(route_fitnesses)))

    #max fitness -> rank 1, 2nd best -> rank 2... worst fitness rank 0
    # sorted_indices = np.argsort(route_fitnesses)[::-1] 
    # print("\n sorted indices", sorted_indices)
    #sorted_routes = [population[index] for index in sorted_indices]
    #print("\n sorted routes", sorted_routes)
    
    sorted_routes_fitnesses = sorted(routes_fitnesses, key=lambda x: x[1])
    #print(sorted_routes_fitnesses)
    sorted_routes = [elt[0] for elt in sorted_routes_fitnesses]
    #print(sorted_routes)
        
    ranks = np.arange(1, len(population)+1)
    #print("\n ranks", ranks)
    total_ranks = sum(ranks)
    
    probabilities = ranks / total_ranks
    #print("/n probabilities", probabilities)
    
    parents = []
    while parents == [] or parents[0] == parents[1]:    
        parents = random.choices(sorted_routes, weights=probabilities, k=2)
    
    return parents[0], parents[1]

In [446]:
parent1, parent2 = selected_parents(population) 
print("parent1", parent1)
print("parent2", parent2)

parent1 ['Tver', 'Zelenograd', 'Khimki', 'Mytishchi', 'Korolyov', 'Shchyolkovo', 'Balashikha', 'Zheleznodorozhnyy', 'Lyubertsy', 'Moscow', 'Odintsovo', 'Podolsk', 'Zhukovskiy', 'Elektrostal', 'Noginsk', 'Orekhovo‐Zuevo', 'Sergiyev Posad', 'Vladimir', 'Kovrov', 'Ivanovo', 'Kostroma', 'Yaroslavl', 'Rybinsk', 'Cherepovets', 'Vologda', 'Dzerzhinsk', 'Nizhny Novgorod', 'Arzamas', 'Murom', 'Ryazan', 'Kolomna', 'Serpukhov', 'Obninsk', 'Kaluga', 'Tula', 'Novomoskovsk', 'Yelets', 'Lipetsk', 'Voronezh', 'Staryy Oskol', 'Belgorod', 'Kursk', 'Oryol', 'Bryansk', 'Smolensk', 'Velikie Luki', 'Pskov', 'Velikiy Novgorod', 'Kolpino', 'Saint Petersburg', 'Petrozavodsk', 'Severodvinsk', 'Arkhangelsk', 'Murmansk', 'Syktyvkar', 'Kirov', 'Yoshkar‐Ola', 'Novocheboksarsk', 'Cheboksary', 'Kazan', 'Ulyanovsk', 'Dimitrovgrad', 'Tolyatti', 'Novokuybyshevsk', 'Samara', 'Syzran', 'Balakovo', 'Engels', 'Saratov', 'Kamyshin', 'Volzhskiy', 'Volgograd', 'Volgodonsk', 'Shakhty', 'Novoshakhtinsk', 'Novocherkassk', 'Rostov

In [ ]:
population = initial_population(10)
parent1, parent2 = selected_parents(population)

print(parent1, parent2)

['Kursk', 'Staryy Oskol', 'Voronezh', 'Lipetsk', 'Yelets', 'Novomoskovsk', 'Tula', 'Serpukhov', 'Obninsk', 'Kaluga', 'Podolsk', 'Odintsovo', 'Moscow', 'Mytishchi', 'Korolyov', 'Shchyolkovo', 'Balashikha', 'Zheleznodorozhnyy', 'Lyubertsy', 'Zhukovskiy', 'Elektrostal', 'Noginsk', 'Orekhovo‐Zuevo', 'Sergiyev Posad', 'Khimki', 'Zelenograd', 'Tver', 'Rybinsk', 'Yaroslavl', 'Kostroma', 'Ivanovo', 'Kovrov', 'Vladimir', 'Murom', 'Arzamas', 'Dzerzhinsk', 'Nizhny Novgorod', 'Cheboksary', 'Novocheboksarsk', 'Yoshkar‐Ola', 'Kazan', 'Ulyanovsk', 'Dimitrovgrad', 'Tolyatti', 'Novokuybyshevsk', 'Samara', 'Syzran', 'Balakovo', 'Engels', 'Saratov', 'Kamyshin', 'Volzhskiy', 'Volgograd', 'Volgodonsk', 'Shakhty', 'Novoshakhtinsk', 'Novocherkassk', 'Rostov‐na‐Donu', 'Bataysk', 'Taganrog', 'Krasnodar', 'Maykop', 'Armavir', 'Stavropol', 'Nevinnomyssk', 'Cherkessk', 'Kislovodsk', 'Pyatigorsk', 'Nalchik', 'Vladikavkaz', 'Nazran', 'Groznyy', 'Khasavyurt', 'Makhachkala', 'Kaspiysk', 'Derbent', 'Astrakhan', 'Elist

In [ ]:
def inver_over(parent1, parent2):
    
    point1, point2 = 0, 0
    while point1 == point2:
        point1 = random.randint(0, len(parent1)-1)
        point2 = random.randint(0, len(parent1)-1)
    #print(point1, point2)
    
    if point1 > point2:
        point1, point2 = point2, point1
        
    inverted_seg = parent1[point1+1:point2+1][::-1]
    #print(inverted_seg)
    offspring = [None] * len(parent1)
    
    offspring[point1+1:point2+1] = inverted_seg
    offspring[point1] = parent1[point1]
    #print("offspring", offspring)
    parent2_index = 0
    for i in range(len(parent2)):
        if offspring[i] is None:
            while parent2[parent2_index] in offspring:
                #print(parent2[parent2_index], parent2_index)
                parent2_index += 1
            offspring[i] = parent2[parent2_index]
            #print("offspring", offspring)
            parent2_index += 1
        
    
    return offspring

In [452]:
optimal_route = None
optimal_distance = float('inf')

population = initial_population(1000)

for step in range(1000):
    #print(f"\nstep {step+1}")
    
    parent1, parent2 = selected_parents(population)
    
    offspring = inver_over(parent1, parent2)
    #offspring_cost = cost(offspring)
    offspring_fitness = fitness(offspring)

    #costs = [(cost(route), route) for route in population]
    fitnesses = [(fitness(route), route) for route in population]
    #min_cost_route = min(costs, key=lambda x:x[0])
    min_fitness_route = min(fitnesses, key=lambda x:x[0])
    #min_cost = min_cost_route[0]
    max_fitness = min_fitness_route[0]
    
    if offspring_fitness > max_fitness:
        index = population.index(min_fitness_route[1])
        population[index] = offspring
        
fitnesses = [fitness(route) for route in population]
max_fitness = max(fitnesses)
print(max_fitness)

KeyboardInterrupt: 

In [451]:
# optimal_route = None
# optimal_distance = float('inf')

# population = initial_population(20)

# for step in range(100):
#     print(f"\nstep {step+1}")
    
#     for i in range(10):
#         parent1, parent2 = selected_parents(population)
#         print("parent1", parent1)
#         print("parent2", parent2)
#         offspring = inver_over(parent1, parent2)
#         population.append(offspring)
#         print(len(population))
    
#     costs = [(cost(route), route) for route in population]
#     min_cost = min(costs)
    
    
#     if offspring_cost < min_cost:
#         index = population.index(min_cost[1])
#         population[index] = offspring

In [449]:
parent1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
parent2 = [4, 6, 1, 8, 2, 5, 3, 9, 7]
#parent2 = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
CHILD = inver_over(parent1, parent2)
CHILD

[6, 2, 5, 4, 3, 1, 8, 9, 7]